# Save & Restore a Model - 保存和读取模型

Save and Restore a model using TensorFlow v2. In this example, we will go over both low and high-level approaches: 
- Low-level: TF Checkpoint.
- High-level: TF Module/Model saver.

使用 TensorFlow v2 保存和读取模型。在这个例子中，我们将介绍两种方法：
- 低级：TF Checkpoint。
- 高级：TF Module/Model saver。

This example is using the MNIST database of handwritten digits as toy dataset
(http://yann.lecun.com/exdb/mnist/).

这个例子使用 MNIST 手写数字数据库作为数据集。

- Author: Aymeric Damien
- Project: https://github.com/aymericdamien/TensorFlow-Examples/

In [72]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np
import keras._tf_keras.keras.optimizers as optimizers

In [73]:
# MNIST dataset parameters. - 数据集超参数
num_classes = 10 # 0 to 9 digits - 数字种类, 0-9共10种
num_features = 784 # 28*28 - 图片大小, 28*28=784 个特征

# Training parameters. - 训练超参数
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

In [74]:
# Prepare MNIST data. - 准备 MNIST 数据以及数据预处理
import keras.api.datasets.mnist as mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Convert to float32. - 转换为 float32 类型（原先为 uint8 类型，代表0~255的灰度值）
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Flatten images to 1-D vector of 784 features (28*28). - 将图像展平为 784 特征（28*28）
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
# Normalize images value from [0, 255] to [0, 1]. - 将图像的值从 [0, 255] 标准化到 [0, 1]
x_train, x_test = x_train / 255., x_test / 255.

In [75]:
# Use tf.data API to shuffle and batch data. - 使用 tf.data API 来打乱 并 分批数据
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

## 1) TF Checkpoint

Basic logistic regression

以一个简单的逻辑回归为例

In [76]:
# Weight of shape [784, 10], the 28*28 image features, and total number of classes.
# 权重矩阵的形状为 [784, 10]，代表 28*28 的图像特征和总类别数.这里只有一层神经网络
W = tf.Variable(tf.random.normal([num_features, num_classes]), name="weight")
# Bias of shape [10], the total number of classes.
# 偏置的形状为 [10]，代表总类别数
b = tf.Variable(tf.zeros([num_classes]), name="bias")

# Logistic regression (Wx + b). - 逻辑回归模型
def logistic_regression(x):
    # Apply softmax to normalize the logits to a probability distribution.
    # 使用 softmax 函数将逻辑回归的输出归一化为概率分布
    return tf.nn.softmax(tf.matmul(x, W) + b)

# Cross-Entropy loss function. - 交叉熵损失函数
def cross_entropy(y_pred, y_true):
    # Encode label to a one hot vector. - 将标签编码为 one-hot 向量,即正确的标签为1，其余为0
    y_true = tf.one_hot(y_true, depth=num_classes)
    # Clip prediction values to avoid log(0) error. - 防止 log(0) 错误
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    # Compute cross-entropy. - 计算交叉熵
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

# Accuracy metric. - 准确率评估方法
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    # 预测结果 是 预测结果向量 中得分最高的项的索引（即 argmax）
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Adam optimizer. - Adam 优化器
optimizer = optimizers.Adam(learning_rate)

In [77]:
# Optimization process.  - 优化过程
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    # 使用梯度带自动计算梯度
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)

        # Compute gradients. - 计算梯度
        gradients = g.gradient(loss, [W, b])

        # Update W and b following gradients. - 根据梯度更新 W 和 b
        optimizer.apply_gradients(zip(gradients, [W, b]))

In [78]:
# Run training for the given number of steps.  - 运行给定步数的训练
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    # 运行优化器更新 W 和 b 的值
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0: # 每固定步数显示一次结果
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 794.371643, accuracy: 0.476562
step: 100, loss: 407.682068, accuracy: 0.714844
step: 150, loss: 271.101624, accuracy: 0.785156
step: 200, loss: 194.569031, accuracy: 0.835938
step: 250, loss: 194.399689, accuracy: 0.835938
step: 300, loss: 199.207306, accuracy: 0.843750
step: 350, loss: 227.731888, accuracy: 0.832031
step: 400, loss: 221.454254, accuracy: 0.839844
step: 450, loss: 131.556885, accuracy: 0.867188
step: 500, loss: 138.140381, accuracy: 0.871094
step: 550, loss: 123.082863, accuracy: 0.910156
step: 600, loss: 129.521027, accuracy: 0.875000
step: 650, loss: 140.932785, accuracy: 0.898438
step: 700, loss: 110.293991, accuracy: 0.894531
step: 750, loss: 113.051117, accuracy: 0.867188
step: 800, loss: 156.307465, accuracy: 0.902344
step: 850, loss: 95.922668, accuracy: 0.906250
step: 900, loss: 70.903198, accuracy: 0.933594
step: 950, loss: 92.833603, accuracy: 0.921875
step: 1000, loss: 49.431381, accuracy: 0.937500


## Save and Load with TF Checkpoint - 使用 TF Checkpoint 保存和读取

In [79]:
# Save weights and optimizer variables. - 保存权重和优化器变量
# Create a dict of variables to save. - 创建一个要保存的变量字典
vars_to_save = {"W": W, "b": b, "optimizer": optimizer} # 这里只有一层, 所以只有 W 和 b
# TF Checkpoint, pass the dict as **kwargs. - TF Checkpoint，将字典作为 **kwargs(关键字参数字典) 传递
checkpoint = tf.train.Checkpoint(**vars_to_save)
# TF CheckpointManager to manage saving parameters. - TF CheckpointManager 用于管理保存参数
saver = tf.train.CheckpointManager(
      checkpoint, directory="./tf-example", max_to_keep=5)

In [80]:
# Save variables. - 保存变量
saver.save()

'./tf-example\\ckpt-1'

In [81]:
# Check weight value. - 看下权重值
np.mean(W.numpy()) # 这里直接看了权重的均值

-0.08043851

In [82]:
# Reset variables to test restore. - 复原W和b变量, 测试从保存的文件中读取恢复
W = tf.Variable(tf.random.normal([num_features, num_classes]), name="weight")
b = tf.Variable(tf.zeros([num_classes]), name="bias")

In [83]:
# Check resetted weight value. - 看下原始的权重值均值
np.mean(W.numpy())

0.009461794

In [84]:
# Set checkpoint to load data. - 创建一个checkpoint对象以加载数据
vars_to_load = {"W": W, "b": b, "optimizer": optimizer}
checkpoint = tf.train.Checkpoint(**vars_to_load)
# Restore variables from latest checkpoint. - 从新的 checkpoint 恢复变量
latest_ckpt = tf.train.latest_checkpoint("./tf-example")
checkpoint.restore(latest_ckpt) # 恢复

In [85]:
# Confirm that W has been correctly restored. - 确认 W 已经正确恢复, 权重值均值和之前一样
np.mean(W.numpy())

-0.08043851

## 2) TF Model

Basic neural network with TF Model

以一个简单的TensorFlow神经网络为例

In [86]:
from keras import Model, layers

In [87]:
# MNIST dataset parameters. - 数据集超参数
num_classes = 10 # 0 to 9 digits - 数字种类, 0-9共10种
num_features = 784 # 28*28 - 图片大小, 28*28=784 个特征

# Training parameters. - 训练超参数
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 100

In [88]:
# Create TF Model. - 创建 TF 模型
class NeuralNet(Model):
    # Set layers. - 初始化层
    def __init__(self):
        super(NeuralNet, self).__init__(name="NeuralNet")
        # First fully-connected hidden layer. - 第一个全连接隐藏层
        self.fc1 = layers.Dense(64, activation=tf.nn.relu)
        # Second fully-connected hidden layer. - 第二个全连接隐藏层
        self.fc2 = layers.Dense(128, activation=tf.nn.relu)
        # Third fully-connecter hidden layer. - 第三个全连接隐藏层
        self.out = layers.Dense(num_classes, activation=tf.nn.softmax)

    # Set forward pass. - 前向传播方法
    def __call__(self, x, is_training=False):
        x = self.fc1(x)
        x = self.out(x)
        if not is_training:
            # tf cross entropy expect logits without softmax, so only
            # apply softmax when not training.
            # tf 交叉熵期望没有 softmax 的 logits，所以只在不训练时应用 softmax
            x = tf.nn.softmax(x)
        return x

# Build neural network model. - 构建神经网络模型
neural_net = NeuralNet()

In [89]:
# Cross-Entropy loss function. - 交叉熵损失函数
def cross_entropy(y_pred, y_true):
    y_true = tf.cast(y_true, tf.int64)
    crossentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
    return tf.reduce_mean(crossentropy)

# Accuracy metric. - 准确率评估方法
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    # 预测结果 是 预测结果向量 中得分最高的项的索引（即 argmax）
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Adam optimizer.
optimizer = tf.optimizers.Adam(learning_rate)

In [90]:
# Optimization process.  - 优化过程
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    # 使用梯度带自动计算梯度
    with tf.GradientTape() as g:
        pred = neural_net(x, is_training=True)
        loss = cross_entropy(pred, y)

        # Compute gradients. - 计算梯度
        gradients = g.gradient(loss, neural_net.trainable_variables)

        # Update W and b following gradients. - 根据梯度更新 W 和 b
        optimizer.apply_gradients(zip(gradients, neural_net.trainable_variables))

In [91]:
# Run training for the given number of steps. - 运行给定步数的训练
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    # 运行优化器更新 W 和 b 的值
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0:
        pred = neural_net(batch_x, is_training=False)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 100, loss: 2.186218, accuracy: 0.917969
step: 200, loss: 2.182905, accuracy: 0.933594
step: 300, loss: 2.180723, accuracy: 0.941406
step: 400, loss: 2.178307, accuracy: 0.960938
step: 500, loss: 2.177688, accuracy: 0.964844
step: 600, loss: 2.182963, accuracy: 0.929688
step: 700, loss: 2.174937, accuracy: 0.980469
step: 800, loss: 2.178718, accuracy: 0.957031
step: 900, loss: 2.176234, accuracy: 0.972656
step: 1000, loss: 2.177423, accuracy: 0.960938


## Save and Load with TF Model - 使用 TF Model 保存和读取

In [92]:
# Save TF model. - 保存 TF 模型
# neural_net.save_weights(filepath="./tfmodel.ckpt") 
# 译者注: 2.17的TensorFlow中需要使用weights.h5的后缀
neural_net.save_weights(filepath="./tfmodel.weights.h5")


In [93]:
# Re-build neural network model with default values.
# 使用默认值重新构建神经网络模型(用于验证保存的模型是否正确)
neural_net = NeuralNet()
# Test model performance.
# 测试模型性能
pred = neural_net(batch_x)
print("accuracy: %f" % accuracy(pred, batch_y))
# 可见准确率几乎为0, 说明模型还没有训练

accuracy: 0.031250


In [94]:
# Load saved weights. - 加载保存的权重
neural_net.load_weights(filepath="./tfmodel.weights.h5")

In [95]:
# Test that weights loaded correctly. - 测试权重是否正确加载
pred = neural_net(batch_x)
print("accuracy: %f" % accuracy(pred, batch_y))
# 可见准确率已经接近1, 说明这个模型是训练过的,读取的结果是正确的

accuracy: 0.960938
